In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

In [45]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(ChromeDriverManager().install())
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    driver.maximize_window()

# =====================================================================================
    # get the login icon and click on it 
# =====================================================================================
    try:
        # get the login button
        login = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'svg[title="login icon"]')))
        if login:
            driver.execute_script("arguments[0].scrollIntoView();", login)
            login.click()
        else:
            pass

    except Exception as e:
        print(f"No login tab found and the error is {e}")

# =====================================================================================
    # complete logiin info
# =====================================================================================

    # get the username tab
    try:
        provided_u_name = "fourbrotherstrading@icloud.com"         
        user_name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'.//input[@id="Email"]')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")

    # get password tab
    try:
        provided_pass = "Muhssan7865"
        password = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/input[@id="Password"]')))
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}")
        
    #  get the terms clicked
    try:
        terms = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/input[@id="IsAgreeToTerms"]'))) 
        if terms:
            terms.click()
        else:
            print(f"No terms found {e}")
    except Exception as e:
        print(f"no terms tab found")
        
    # get the login tab to enter the username and password
    try:
        # login_tab_css = "submit"
        login_tab = driver.find_element(By.XPATH,  './/button[@type="submit"]')
        driver.execute_script("arguments[0].scrollIntoView();", login_tab)
        login_tab.click()
    except Exception as e:
        print(f"No login tab found and the error is {e}")

# =====================================================================================
    # handle cookies
# =====================================================================================
    try:
        cookie = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@class="cky-btn cky-btn-reject"]')))
        if cookie:
            cookie.click()
        else:
            pass
    except Exception as e:
        print("No cookies tab found and the error is {e}")
    
    # handle the subscribe tab
    try:
        sub = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@class="close js-mls-close"]')))
        if sub:
            sub.click()
        else:
            pass
    except Exception as e:
        print("No subscribe tab")

# =====================================================================================
    # get totaal cars
# =====================================================================================
    try:
        num_cars = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//p[@class="pager__label"])[1]//span'))).text.strip()
        if num_cars:
            cars_list = num_cars.split(" ")
            cars =  int(cars_list[1]) # get only number
            print(f"Total {cars} of cars found")
    except Exception as e:
        print(f"No number of cars found and error is {e}")

# # =====================================================================================
#     # get the car link and click on it to proceed to main car information page
# # =====================================================================================
    try:
        link = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="ui-listings__grid-item"])[1]')))
        if link:
            driver.execute_script("arguments[0].scrollIntoView();", link)
            link.click()
        else:
            pass
    except Exception as e:
        print(f"No link found and the error is {e}")
# ===================================================================================== 

    
    results = [] # to append the dict values later on
    for i in range(cars):# we will make use of number of pages here

    # ===================================================================================== 
        # get the name of the car
    # =====================================================================================
        details = {}

        # title --> done
        try:
            # car_name_css = ".row h2"
            car_name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="lot-title-header__item"]//h1')))
            if car_name:
                details["Title"] = car_name.text.strip()
            else:
                details["Title"] = "No car name for this car found"
        except Exception as e:
            print(f"No car name found and the error is {e}")
            
        # lot --> done
        try: 
            lot = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[@class="auction-header__item--lot-title--lotno"]'))).text.strip()
            if lot:
                details['Lot'] = lot.split(" ")[1]
            else:
                details['Lot'] = 'na'
        except Exception as e:
            print("No lot number")
        
        # auc end dat and time --> done
        try:
            auc_end = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@class="lot-title-header__ends"]'))).text.strip()
            if auc_end:
                acu_data_time = auc_end.split(": ")
                details[acu_data_time[0]] = acu_data_time[1]
            else:
                print(*"No auc end details")
        except Exception as e:
            print("No auc end date and time")

        # other information
        try:
            info_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/ul[@class="listing-properties__list"]'))) 
            if info_card:
                info_rows = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/li[contains(@class,"listing-properties__list-item")]')))
                if info_rows:
                    for info_row in info_rows:
                        info_lbl = info_row.find_element(By.XPATH, './/p[@class="listing-properties__name"]').text.strip()
                        info_val = info_row.find_element(By.XPATH, './/p[@class="listing-properties__value"]').text.strip()
                        if info_lbl and info_val:
                            details[info_lbl] = info_val
                        else:
                            print("No details")
        except Exception as e:
            print("No info card found")
        
        # append the in the results 
        results.append(details)

        # images
        try:
            for _ in range(2):  # Adjust range for deeper scrolling
                driver.execute_script("window.scrollBy(0, 500);")  # Scroll down 500px
                time.sleep(1)  # Wait for content to load
            clk_img =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@class="btn btn--lot-gallery btn--text-case"]'))) 
            if clk_img:

                driver.execute_script("arguments[0].scrollIntoView();", clk_img)
                clk_img.click()
                imgs_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@id="gallery-modal-images"]'))) 
                imgs = WebDriverWait(imgs_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class="gallery__modal-gallery-image"]')))  
                if imgs:
                    imgs_lst = [img.get_attribute("src") for img in imgs]
                    details['Images'] = ", ".join(imgs_lst)
                else:
                    print("No images found")

                    clck_back = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@id="gallery-modal-close"]'))) 
                    driver.execute_script("arguments[0].scrollIntoView();", clck_back)
                    clck_back.click()
        except Exception as e:
            print(f"No image found {e}")
        # =====================================================================================
        # get the next button
    # =====================================================================================
        try:
            next_button = driver.find_element(By.XPATH, './/a[@class="btn-next-lot btn btn--lot btn--inline btn--title-case "]')
            if next_button:
                next_button.click()
            else:
                break
        except Exception as e:
            print("No next button found hence no more cars are available")

# =====================================================================================
    # create a dataframe 
    df = pd.DataFrame.from_dict(results)
    df.to_csv("brightwells.csv", index=False)
    driver.quit()

path = "https://www.brightwells.com/timed-sale/5753"
scrape(path)

Total 59 of cars found
No info card found
No info card found
No info card found
No image found Message: element not interactable
  (Session info: chrome=133.0.6943.127)
Stacktrace:
	GetHandleVerifier [0x01080283+25139]
	(No symbol) [0x0100B234]
	(No symbol) [0x00EE0300]
	(No symbol) [0x00F29454]
	(No symbol) [0x00F1DE55]
	(No symbol) [0x00F4CF3C]
	(No symbol) [0x00F1D8E4]
	(No symbol) [0x00F4D1B4]
	(No symbol) [0x00F6E6E1]
	(No symbol) [0x00F4CD36]
	(No symbol) [0x00F1BD29]
	(No symbol) [0x00F1D064]
	GetHandleVerifier [0x0138B143+3215603]
	GetHandleVerifier [0x013A22BA+3310186]
	GetHandleVerifier [0x0139C4D2+3286146]
	GetHandleVerifier [0x01119C80+654384]
	(No symbol) [0x010145BD]
	(No symbol) [0x010114A8]
	(No symbol) [0x01011647]
	(No symbol) [0x01003D20]
	BaseThreadInitThunk [0x7552FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x771E82AE+286]
	RtlGetAppContainerNamedObjectPath [0x771E827E+238]

No info card found
No info card found
No info card found
No image found Message: element c

In [3]:
df=pd.read_csv("brightwells.csv")
df.head()

,Title,Lot,Auction Ends,Year,Registration Number,Imported Status,Engine Size,Transmission,Mileage/Hours,Colour,...,Registered Owners,VAT Status,Buyers Premium (excl. VAT),Images,CAP Average,CAP Clean,Chassis Number,Lot Location,Documents,Comments & Features
0,HYUNDAI TUCSON SE NAV CRDI 2WD\nEstate,8,Thu 27th Feb @ 10:01 am,2018.0,BG68XMK,No,1598cc,6 Speed Manual Diesel,60593 miles Warranted,Grey,...,2.0,No VAT,"7.5%, Minimum £200",https://db9n4su8pesko.cloudfront.net/eyJidWNrZ...,NaN,NaN,NaN,NaN,NaN,NaN
1,KIA XCEED 2 ISG\n5 Door Hatchback,9,Thu 27th Feb @ 10:02 am,2020.0,RO70RMU,No,998cc,6 Speed Manual Petrol,59854 miles Warranted,Silver,...,2.0,Inclusive of VAT,"7.5%, Minimum £200",https://db9n4su8pesko.cloudfront.net/eyJidWNrZ...,NaN,NaN,NaN,NaN,NaN,NaN
2,NISSAN MICRA TEKNA IG-T\n5 Door Hatchback,10,Thu 27th Feb @ 10:03 am,2022.0,OW72EAC,No,999cc,5 Speed Manual Petrol,6949 miles Warranted,Grey,...,1.0,Inclusive of VAT,"7.5%, Minimum £200",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FORD FOCUS ZETEC 100\n5 Door Hatchback,11,Thu 27th Feb @ 10:04 am,2011.0,NJ11MWF,No,1596cc,5 Speed Manual Petrol,75798 miles Warranted,Black,...,3.0,No VAT,"7.5%, Minimum £200",https://db9n4su8pesko.cloudfront.net/eyJidWNrZ...,NaN,NaN,NaN,NaN,NaN,NaN
4,FORD FOCUS ZETEC CLIMATE\n5 Door Hatchback,12,Thu 27th Feb @ 10:05 am,2007.0,FL07EKF,No,1798cc,5 Speed Manual Petrol,112905 miles Warranted,Silver,...,4.0,No VAT,"7.5%, Minimum £200",https://db9n4su8pesko.cloudfront.net/eyJidWNrZ...,NaN,NaN,NaN,NaN,NaN,NaN


### image downloader

In [4]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

reg_img = df[["Registration Number", "Images"]]

def download_images(data, main_folder="Images"): 
    os.makedirs(main_folder, exist_ok=True)
    
    for index, row in data.iterrows():
        reg_no = row["Registration Number"] 
        image_urls = row["Images"]

        # Check if 'Images' column is empty or NaN
        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")  # Split URLs by comma
        
        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)
        
        for idx, url in enumerate(image_urls):
            url = url.strip()
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url) 
            
            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") 
                continue
            
            try:
                response = requests.get(url, stream=True) 
                response.raise_for_status()
                
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                file_extension = file_name.split(".")[-1]
                
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"
                
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# Call the function
download_images(reg_img)

Downloaded: Images\BG68XMK\BG68XMK_1_image_1.jpg
Downloaded: Images\BG68XMK\BG68XMK_2_image_2.jpg
Downloaded: Images\BG68XMK\BG68XMK_3_image_3.jpg
Downloaded: Images\BG68XMK\BG68XMK_4_image_4.jpg
Downloaded: Images\BG68XMK\BG68XMK_5_image_5.jpg
Downloaded: Images\BG68XMK\BG68XMK_6_image_6.jpg
Downloaded: Images\BG68XMK\BG68XMK_7_image_7.jpg
Downloaded: Images\BG68XMK\BG68XMK_8_image_8.jpg
Downloaded: Images\BG68XMK\BG68XMK_9_image_9.jpg
Downloaded: Images\BG68XMK\BG68XMK_10_image_10.jpg
Downloaded: Images\BG68XMK\BG68XMK_11_image_11.jpg
Downloaded: Images\BG68XMK\BG68XMK_12_image_12.jpg
Downloaded: Images\BG68XMK\BG68XMK_13_image_13.jpg
Downloaded: Images\BG68XMK\BG68XMK_14_image_14.jpg
Downloaded: Images\BG68XMK\BG68XMK_15_image_15.jpg
Downloaded: Images\BG68XMK\BG68XMK_16_image_16.jpg
Downloaded: Images\BG68XMK\BG68XMK_17_image_17.jpg
Downloaded: Images\BG68XMK\BG68XMK_18_image_18.jpg
Downloaded: Images\BG68XMK\BG68XMK_19_image_19.jpg
Downloaded: Images\BG68XMK\BG68XMK_20_image_20.jp